In [1]:
from hangul_utils import split_syllables, join_jamos
from khaiii import KhaiiiApi
api = KhaiiiApi()

#요에서 는가 빼줘야됨
요 = ['을까', 'ㄹ까', '지', '니까', '으니까', '어', '아', '는데', 'ㄴ가', 'ㄹ게', '래', '잖아', '네', '여', 'ㄹ래', '게', '나', '군', '을게', '고', '다고', '지만', '은데', 'ㄴ지', '는지', '라고', 'ㄴ데', '는걸', '아서', '거든', '더라고', '어서', '려고', '을래', '대', 'ㄴ대', '던데', '더군', '여서', '건가', '어야지', 'ㄹ걸', 'ㄹ께', '아야지', '다면서', '는군', 'ㄴ다고', '은가', '다니', '다면', '실까', '가', '신가', '서', '까']
나머지 = ['습니까', '자', 'ㅂ니까', '야', '라', '에요', '죠', '요', '세요', '니', '려요', '던가요', '나아요', '오', '라서요', 'ㅂ시오', '아라', '구나', '는구나', '냐', '더라', '어야죠', '느냐', '렴', '거에요', '예요', '어라', '다네요', '는가','아야죠']
다 = ['다', '습니다', 'ㅂ니다', '니다', 'ㄴ다', '답니다', '는다', 'ㄴ답니다', 'ㅂ시다', '느냐다', '단다', '서다', '란다', 'ㄴ단다', '시다', '랍니다', '읍시다'] 
yoo_tag = ['VA', 'VX', 'XSA', 'XSV', 'JKQ', 'VCP', 'ZN']


##Use in Changer
def to2lists(input):
    lis_word = []
    lis_tag = []
    analyzed = api.analyze(input)  
    for data in analyzed:
        for morph in data.morphs:
            lis_word.append(morph.lex)
            lis_tag.append(morph.tag)
    return lis_word, lis_tag

## Main Class ## 
class Changer(object):
    
    ## High -> Low ##    
    def low_high(self, stc):
        result = stc
        
        lis_word, lis_tag = to2lists(result)
        print(lis_word)
        print(lis_tag)
        if lis_tag[-2] == 'EF' and result[-3:-1] != '니다' and result[-3:-1] != '시다' and result[-2] != '요':
            #요를 붙였다 땟다해서 처리가능한 애들
            if lis_word[-2] in 요:
                result = result[:-1] + '요.'
            #다로 끝나는 애들
            elif lis_word[-2] in 다:
                jamo = split_syllables(result[-3])
                #는다 -> 습니다
                if lis_word[-2] == '는다':    
                    result = result[:-3] + '습니다.'
                #ㄹ,ㄴ 받침 
                elif jamo[-1] == 'ㄹ' or jamo[-1] == 'ㄴ':
                    result = result[:-3] + join_jamos(jamo[:-1]+'ㅂ')+'니다.'
                #받침 없음 -> ㅂ니다
                elif jamo[-1]< 'ㄱ' or jamo[-1]>'ㅎ':
                    result = result[:-3] + join_jamos(jamo+'ㅂ')+'니다.'
                #받침있는 경우
                else:
                    result = result[:-2] + '습니다.'
            #딱히 규칙없는 친구들
            elif lis_word[-2] in 나머지:
                #구나
                if '구나' in lis_word[-2]:
                    result = result[:-len(lis_word[-2])-1] + '네요.'
                #어라, 아라
                elif lis_word[-2] == '아라' or lis_word[-2] == '어라':
                    last_word = result[-(len(lis_word[-3])+len(lis_word[-2])+1):-1]
                    jamo = split_syllables(lis_word[-3])
                    if lis_word[-3]+lis_word[-2] == last_word:
                        if jamo[-1] == 'ㄹ':
                            result = result[:-len(last_word)-1] + join_jamos(jamo[:-1]) + '세요.'
                        elif jamo[-1] >= 'ㄱ' and jamo[-1] <= 'ㅎ':
                            result = result[:-len(last_word)-1] + lis_word[-3] + '으세요.'
                        else:
                            result = result[:-len(last_word)-1] + lis_word[-3] + '세요.'
                    elif result[-3:-1] == '라라':
                        result = result[:-len(last_word)-1] + join_jamos(jamo[:-1]) + '르세요.'
                    else:
                        if jamo[-1] == 'ㄹ':
                            result = result[:-len(last_word)] + join_jamos(jamo[:-1]) + '세요.'
                        elif jamo[-1] >= 'ㄱ' and jamo[-1] <= 'ㅎ':
                            result = result[:-len(last_word)] + lis_word[-3] + '으세요.'
                        else:
                            result = result[:-len(last_word)] + lis_word[-3] + '세요.'
                #렴
                elif lis_word[-2] == '렴':
                    jamo = split_syllables(lis_word[-3])
                    if jamo[-1] == 'ㄹ':
                        result = result[:-3] + join_jamos(jamo[:-1]) + '세요.'
                    else:
                        result = result[:-2] + '세요.'
                #야
                elif lis_word[-2] == '야':
                    result = result[:-2] + '에요.'
                #니 냐 느냐
                elif lis_word[-2] == '니' or lis_word[-2] == '냐' or lis_word[-2] == '느냐' or lis_word[-2] == '는가':
                    result = result[:-len(lis_word[-2])-1] + '나요.'
                #자
                elif lis_word[-2] == '자':
                    result = result[:-2] + '지요.'
                #이거해야됨!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                elif lis_word[-2] == '더라':
                    jamo = split_syllables(result[-4])
                    if jamo[1] == 'ㅏ' or jamo[1] == 'ㅗ':
                        if result[-4] == '하':
                            result = result[:-3] + '여요.'
                        else:
                            result = result[:-3] + '아요.'
                    else:
                        result = result[:-3] + '어요.'
                #라
                elif lis_word[-2] == '라':
                    jamo = split_syllables(result[-3])
                    if lis_tag[-3] == 'VV':
                        if jamo[-1] <= 'ㅎ' and jamo[-1] >= 'ㄱ':
                            result = result[:-1] + '요.'
                        else:
                            result = result[:-2] + '세요.'
                    elif lis_tag[-3] == 'VCN' or lis_tag[-3] == 'VA' or lis_tag[-3] == 'NNB':
                        reulst = result[:-1] + '요.'
                    elif lis_tag[-3] == 'VX' or lis_tag[-3] == 'XSV':
                        if lis_word[-3] == '달':
                            reulst = result[:-3] + '주세요.'
                        elif jamo[-1] == 'ㄹ':
                            result = result[:-3] + join_jamos(jamo[:-1]) + '세요.'
                        else:
                            reulst = result[:-1] + '요.'
                    
        elif lis_tag[-2] in yoo_tag:
            result = result[:-1] + '요.'
        elif lis_tag[-2] == 'ETN':
            if lis_word[-2] == '음':
                result = result[:-2] + '습니다.'
            elif lis_word[-2] == 'ㅁ':
                jamo = split_syllables(result[-2])
                result = result[:-2] + join_jamos(jamo[:-1]+'ㅂ') + '니다.'
        elif lis_tag[-2] == 'VV':
            if lis_word[-2] == '함' or lis_word[-2] == '줌':
                jamo = split_syllables(result[-2])
                result = result[:-2] + join_jamos(jamo[:-1]+'ㅂ') + '니다.'
            else:
                result = result[:-1] + '요'
        elif lis_tag[-2] == 'EC':
            if lis_word[-2] == '야':
                result = result[:-2] + '에요.'
            elif lis_word[-2] == '니':
                result = result[:-2] + '나요.'
            elif lis_word[-2] == '다':
                jamo = split_syllables(result[-3])
                #는다 -> 습니다
                if lis_word[-2] == '는다':    
                    result = result[:-3] + '습니다.'
                #ㄹ,ㄴ 받침 
                elif jamo[-1] == 'ㄹ' or jamo[-1] == 'ㄴ':
                    result = result[:-3] + join_jamos(jamo[:-1]+'ㅂ')+'니다.'
                #받침 없음 -> ㅂ니다
                elif jamo[-1]< 'ㄱ' or jamo[-1]>'ㅎ':
                    result = result[:-3] + join_jamos(jamo+'ㅂ')+'니다.'
                #받침있는 경우
                else:
                    result = result[:-2] + '습니다.'
            else:
                result = result[:-1] + '요.'
        return result
    
    
    def processText(self,stc):
        result = stc
        
        flag = 0
        if result[-1]=='\n':
            result = result.replace('\n','')
        if result[-1] =='?' or result[-1] =='.' or result[-1] =='!' or result[-1] =='\"':
            result = result
        else:
            result = result+'.'
            flag = 1

        res = self.low_high(result)
        
        if flag ==1:
            res = res[:-1]
    
        return res
    
    


In [2]:
ch = Changer()

a = '이거 진짜 알더라.'
cc = ch.processText(a)
print(cc)


['이거', '진', '짜', '알', '더라', '.']
['NP', 'NNG', 'MAG', 'VV', 'EF', 'SF']
이거 진짜 알아요.
